In [2]:
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dataclasses import dataclass
warnings.filterwarnings("ignore")


seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

NameError: name 'random' is not defined

## Config

In [21]:
DATASET_PATH = "../data/raw/heart_cleveland_upload.csv"
TARGET_NAME = "condition"

# Load data

In [22]:
train = pd.read_csv(DATASET_PATH)

In [23]:
X = train.drop(TARGET_NAME, axis=1)
y = train[TARGET_NAME]

In [58]:
from sklearn.covariance import EllipticEnvelope
from sklearn.decomposition import PCA
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    IsolationForest,
    RandomForestClassifier,
)
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import (
    KFold,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    learning_curve,
    train_test_split,
)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [59]:
gradclass = GradientBoostingClassifier(random_state=seed)
knclass = KNeighborsClassifier()
dectree = DecisionTreeClassifier(random_state=seed)
svc = SVC()
randfclass = RandomForestClassifier(random_state=seed)
adaclass = AdaBoostClassifier(random_state=seed)
mlpclass = MLPClassifier(random_state=seed)
gsclass = GaussianNB()

In [60]:
cv = KFold(5, shuffle=True, random_state=seed)
classifiers = [
    gradclass,
    knclass,
    dectree,
    svc,
    randfclass,
    adaclass,
    mlpclass,
    gsclass,
]

In [1]:
!pip install dataclasses

In [13]:
# @dataclass()
# class TrainingPipelineParams:
#     input_data_path: str
#     output_model_path: str
#     metric_path: str
#     splitting_params: SplittingParams
#     feature_params: FeatureParams
#     train_params: TrainingParams
#     downloading_params: Optional[DownloadParams] = None
#     use_mlflow: bool = False
#     mlflow_uri: str = "http://18.156.5.226/"
#     mlflow_experiment: str = "inference_demo"

@dataclass()
class LR_Params:
    a: list
    b: list
    
@dataclass()
class Params:
    a: int
    b: str
    lr: LR_Params


In [14]:
p = Params(
    1,
    '3',
    LR_Params(
        [1,44,5],
        ["a","b"]
    )
)

In [15]:
p.__dict__

{'a': 1, 'b': '3', 'lr': LR_Params(a=[1, 44, 5], b=['a', 'b'])}

In [10]:
dir(p)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'a',
 'b',
 'c',
 'e']

In [61]:
def model_check(X, y, classifiers, cv):

    """A function for testing multiple classifiers and return several metrics."""

    model_table = pd.DataFrame()

    row_index = 0
    for cls in classifiers:

        MLA_name = cls.__class__.__name__
        model_table.loc[row_index, "Model Name"] = MLA_name

        cv_results = cross_validate(
            cls,
            X,
            y,
            cv=cv,
            scoring=("accuracy", "f1", "roc_auc"),
            return_train_score=True,
            n_jobs=-1,
        )
        model_table.loc[row_index, "Train Roc/AUC Mean"] = cv_results[
            "train_roc_auc"
        ].mean()
        model_table.loc[row_index, "Test Roc/AUC Mean"] = cv_results[
            "test_roc_auc"
        ].mean()
        model_table.loc[row_index, "Test Roc/AUC Std"] = cv_results[
            "test_roc_auc"
        ].std()
        model_table.loc[row_index, "Train Accuracy Mean"] = cv_results[
            "train_accuracy"
        ].mean()
        model_table.loc[row_index, "Test Accuracy Mean"] = cv_results[
            "test_accuracy"
        ].mean()
        model_table.loc[row_index, "Test Acc Std"] = cv_results["test_accuracy"].std()
        model_table.loc[row_index, "Train F1 Mean"] = cv_results["train_f1"].mean()
        model_table.loc[row_index, "Test F1 Mean"] = cv_results["test_f1"].mean()
        model_table.loc[row_index, "Test F1 Std"] = cv_results["test_f1"].std()
        model_table.loc[row_index, "Time"] = cv_results["fit_time"].mean()

        row_index += 1

    model_table.sort_values(by=["Test F1 Mean"], ascending=False, inplace=True)

    return model_table

In [62]:
raw_models = model_check(X, y, classifiers, cv)
display(raw_models)

,Model Name,Train Roc/AUC Mean,Test Roc/AUC Mean,Test Roc/AUC Std,Train Accuracy Mean,Test Accuracy Mean,Test Acc Std,Train F1 Mean,Test F1 Mean,Test F1 Std,Time
7,GaussianNB,0.911066,0.904411,0.035803,0.856909,0.838418,0.053549,0.839788,0.825296,0.054897,0.004890
6,MLPClassifier,0.927190,0.902206,0.058272,0.850190,0.828475,0.069370,0.828321,0.817971,0.054025,0.756070
4,RandomForestClassifier,1.000000,0.905921,0.039319,1.000000,0.831808,0.058496,1.000000,0.811806,0.055205,0.628063
0,GradientBoostingClassifier,0.999957,0.885011,0.055375,0.995791,0.801299,0.052370,0.995487,0.780407,0.050809,0.461711
5,AdaBoostClassifier,0.986853,0.867675,0.071706,0.931826,0.784407,0.068264,0.925483,0.761896,0.071321,0.400489
2,DecisionTreeClassifier,1.000000,0.760465,0.062587,1.000000,0.760621,0.063644,1.000000,0.735606,0.066903,0.005708
1,KNeighborsClassifier,0.847873,0.685415,0.058528,0.772701,0.646780,0.058285,0.747835,0.596281,0.068546,0.002924
3,SVC,0.763246,0.738365,0.078969,0.679293,0.656723,0.046699,0.581585,0.550965,0.057644,0.007954


## Finetuning

In [32]:
def hyperparameter_tune(base_model, parameters, n_iter, cv, X, y):

    """A function for optimizing mutliple classifiers"""

    start_time = time.time()
    optimal_model = RandomizedSearchCV(
        base_model,
        param_distributions=parameters,
        n_iter=n_iter,
        cv=cv,
        scoring="f1",
        n_jobs=-1,
        random_state=seed,
    )

    optimal_model.fit(X, y)

    scores = cross_val_score(optimal_model, X, y, cv=cv, n_jobs=-1, scoring="f1")
    stop_time = time.time()

    print("====================")
    print(f"Updated Parameters for {str(base_model.__class__.__name__)}")
    print(
        "Cross Val Mean: {:.3f}, Cross Val Stdev: {:.3f}".format(
            scores.mean(), scores.std()
        )
    )
    print("Best Score: {:.3f}".format(optimal_model.best_score_))
    print("Best Parameters: {}".format(optimal_model.best_params_))
    print(
        "Elapsed Time:", time.strftime("%H:%M:%S", time.gmtime(stop_time - start_time))
    )
    print("====================")

    return optimal_model.best_params_, optimal_model.best_score_


def fine_tune_models(models=[randfclass, svc, adaclass]):
    best_models, best_scores = [], []
    # fine tuning models
    for model in models:
        if model == svc:
            parameters = {
                "C": [1, 3, 9, 27],
                "tol": [1e-2, 1e-3, 1e-4],
                "kernel": ["linear", "rbf", "sigmoid"],
                "shrinking": [True, False],
            }
        if model == randfclass:
            parameters = {
                "max_depth": [2, 3, 5],
                "n_estimators": [50, 100, 150],
                "criterion": ["gini", "entropy"],
                "bootstrap": [True, False],
            }
        if model == adaclass:
            parameters = {
                "base_estimator": [None, dectree],
                "n_estimators": [50, 100, 150],
                "algorithm": ["SAMME", "SAMME.R"],
                "learning_rate": [0.8, 1, 1.2],
            }
        best_params, best_score = hyperparameter_tune(
            model, parameters, 20, cv, X, y
        )
        model.set_params(**best_params)
        best_models.append(model)
        best_scores.append(best_score)
    return best_models, best_scores

In [34]:
best_models, best_scores = fine_tune_models()

Updated Parameters for RandomForestClassifier
Cross Val Mean: 0.803, Cross Val Stdev: 0.058
Best Score: 0.827
Best Parameters: {'n_estimators': 100, 'max_depth': 5, 'criterion': 'entropy', 'bootstrap': False}
Elapsed Time: 00:02:20
Updated Parameters for SVC
Cross Val Mean: 0.821, Cross Val Stdev: 0.052
Best Score: 0.825
Best Parameters: {'tol': 0.001, 'shrinking': False, 'kernel': 'linear', 'C': 1}
Elapsed Time: 00:04:48
Updated Parameters for AdaBoostClassifier
Cross Val Mean: 0.820, Cross Val Stdev: 0.050
Best Score: 0.820
Best Parameters: {'n_estimators': 50, 'learning_rate': 0.8, 'base_estimator': None, 'algorithm': 'SAMME'}
Elapsed Time: 00:01:12


In [35]:
best_scores

[0.8268514801433309, 0.8252844457966356, 0.8200040742623423]

In [64]:
best_model = best_models[np.argmax(best_scores)]

In [69]:
best_model.fit(X,y)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=5,
                       random_state=42)

In [70]:
best_model.predict(X)

array([0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0.,
       1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
       1., 1., 0., 1., 1.

In [72]:
import pickle
FILENAME = 'MODEL'

In [75]:
with open(FILENAME,'wb') as file:
    pickle.dump(best_model, file)

In [76]:
with open(FILENAME,'rb') as file:
    model = pickle.load(file)

In [77]:
model

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=5,
                       random_state=42)

In [ ]:
model.predict(X)

array([0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0.,
       1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
       1., 1., 0., 1., 1.